In [69]:
import json
import pandas as pd
import numpy as np
import PySAM.Pvwattsv8 as PVWatts
import PySAM.Grid as Grid
import csv
import matplotlib.pyplot as plt
from datetime import datetime
from matplotlib.dates import DateFormatter
import geopandas as gpd
import math

In [70]:
def loadModules(file_names):
    pv = PVWatts.new()
    grid = Grid.from_existing(pv)

    with open(file_names[0], 'r') as file:
        data = json.load(file)
        for k, v in data.items():
                pv.value(k, v)

    with open(file_names[1], 'r') as file:
        data = json.load(file)
        for k, v in data.items():
                grid.value(k, v)

    return pv, grid

file_names = ["/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/Scripts/sunEstimation/pysam_template_pvwattsv8.json",
    "/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/Scripts/sunEstimation/pysam_template_grid.json"]

loadModules(file_names)

(<Pvwattsv8 at 0x7fa261169740>, <Grid at 0x7fa2518dae80>)

In [71]:
planeFile = "/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/Results/Test_70_el Besòs i el Maresme/Parcels/4054901DF3845C/408/Shading/1.csv"
shadingProfilesDF = pd.read_csv(planeFile, header=None)
tilts = shadingProfilesDF.iloc[0][3:363]


def get_matrix(tilts):
    tilts = tilts.round()
    angles = np.unique(tilts)
    azimuths = np.arange(0, 360, 1)

    matrix = []
    singleRow = []
    singleRow.append(0)
    for angle in azimuths:
        singleRow.append(angle)

    matrix.append(singleRow)
    for i, angle in enumerate(angles):
        singleRow = []
        singleRow.append(angle)
        for j, tilt in enumerate(tilts):
            if angle <= tilt:
                singleRow.append(100)
            else:
                singleRow.append(0)
        matrix.append(singleRow)

    # matrix = np.zeros((len(angles) + 1, len(azimuths) + 1), dtype=int)
    # matrix[0, 1:] = azimuths
    # matrix[1:, 0] = angles

    # for i, angle in enumerate(angles):
    #     for j, tilt in enumerate(tilts):
    #         if angle <= tilt:
    #             matrix[i + 1, j + 1] = 100

    return matrix

shadingMatrix = get_matrix(tilts)
shadingMatrix

[[0,
  0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,

In [46]:
def readShadingMatrix(shadingMatrixPath):
    """
    Returns the shadingMatrix from the given matrixFile (csv)
    """
    shadingMatrix = []

    with open(shadingMatrixPath) as csvfile:
        reader = csv.reader(csvfile)
        for row in reader:
            singleRow = []
            for element in row:
                if(element == ''):
                    singleRow.append(0.0)
                else:    
                    singleRow.append(float(element))
            shadingMatrix.append(singleRow)

    return shadingMatrix

ZlinMatrix = readShadingMatrix("/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/Scripts/sunEstimation/SampleMatrixZlin.csv")
ZlinMatrix

[[0.0,
  60.0,
  63.0,
  66.0,
  69.0,
  72.0,
  75.0,
  78.0,
  81.0,
  84.0,
  87.0,
  90.0,
  93.0,
  96.0,
  99.0,
  102.0,
  105.0,
  108.0,
  111.0,
  114.0,
  117.0,
  120.0,
  123.0,
  126.0,
  129.0,
  132.0,
  135.0,
  138.0,
  141.0,
  144.0,
  147.0,
  150.0,
  153.0,
  156.0,
  159.0,
  162.0,
  165.0,
  168.0,
  171.0,
  174.0,
  177.0,
  180.0,
  183.0,
  186.0,
  189.0,
  192.0,
  195.0,
  198.0,
  201.0,
  204.0,
  207.0,
  210.0,
  213.0,
  216.0,
  219.0,
  222.0,
  225.0,
  228.0,
  231.0,
  234.0,
  237.0,
  240.0,
  243.0,
  246.0,
  249.0,
  252.0,
  255.0,
  258.0,
  261.0,
  264.0,
  267.0,
  270.0,
  273.0,
  276.0,
  279.0,
  282.0,
  285.0,
  288.0,
  291.0,
  294.0,
  297.0,
  300.0],
 [0.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,
  100.0,


In [72]:
planeIdGDF = gpd.read_file("/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/Results/Test_70_el Besòs i el Maresme/Parcels/4054901DF3845C/408/Plane Identification/408.gpkg")
plane = planeIdGDF[planeIdGDF.cluster == 1]

def getInfoRoof(plane):
    tilt = plane.tilt.values[0]
    azimuth = plane.azimuth.values[0]
    area = plane.area.values[0]/math.cos(tilt*math.pi/180)
    return  area, tilt, azimuth
    
area, tilt, azimuth = getInfoRoof(plane)
plane

,cluster,A,B,D,tilt,azimuth,silhouette,geometry
2,1,-0.237036,0.244355,-1.017541e+06,18.8,135.871064,0.850265,"POLYGON ((433933.943 4585214.663, 433933.943 4..."


In [79]:
def runPySAMSimulation(file_names, tilts, plane, tmyfile, exportPath):
    pv, grid = loadModules(file_names)
    shadingMatrix = get_matrix(tilts)

    area, tilt, azimuth = getInfoRoof(plane)
    
    ratio=0.400/(1.879*1.045)

    modifiedParams = {"shading_azal": shadingMatrix,
        "system_capacity": area*ratio, #*self.pv.value("gcr"), #We don't need the area by the ground coverage ratio
        "tilt": tilt,
        "azimuth": azimuth,
        "solar_resource_file": tmyfile}

    for i in range(len(modifiedParams)): 
        pv.value(list(modifiedParams.keys())[i], list(modifiedParams.values())[i])

    modules = [pv, grid]
    
    for m in modules:
        m.execute()

    generation = pv.export()["Outputs"]["ac"]
    generation = np.array(generation).reshape(365, 24)
    generation_df = pd.DataFrame(generation)
    
    # generation_df.to_csv((exportPath + '.csv'), index=False)
    # return generation_df/area
    return pv.export()["Outputs"]

planeFile = "/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/Results/Test_70_el Besòs i el Maresme/Parcels/4054901DF3845C/408/Shading/1.csv"
shadingProfilesDF = pd.read_csv(planeFile, header=None)
tilts = shadingProfilesDF.iloc[3][3:363]


tmyfile = "/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/RAW_Data/TMY/NREL/419806_41.41_2.22_tmy-2022.csv"
generation = runPySAMSimulation(file_names, tilts, plane, tmyfile, "exportPath")

print(sum(generation["poa"]), generation["annual_energy"])

1901355.8404767185 37413.211652256345


In [34]:
np.max(shadingProfilesDF.iloc[0][3:363])

28.0

In [27]:
print(sum(generation["poa"]), generation["aoi"])

1729310.0062196068 (0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 70.19096787692747, 61.072645943482264, 54.03088867082907, 50.2500758329923, 50.49578508636525, 54.71780587203749, 62.106173377234036, 71.67668109578393, 82.6418268974297, 94.13727102786672, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 70.20822872613552, 61.0672282483013, 53.99339105462269, 50.17634725316068, 50.38985518187358, 54.590745819369495, 61.969185028902025, 71.53713432789422, 82.50406685506223, 94.02199920394156, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 70.21653228411128, 61.05449316169909, 53.948113236263325, 50.094673082974374, 50.27635539452227, 54.45698863704555, 61.826540984289544, 71.39273423327067, 82.3616262864258, 93.89807772310843, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 70.2282645338374, 61.03734331859372, 53.896999246851095, 50.00650180028192, 50.15651691572909, 54.31781729345427, 61.67975899557658, 71.24570372244413, 82.2188421

In [65]:
generation["Shading"].keys()

dict_keys(['shading_azal', 'shading_en_azal', 'shading_en_diff', 'shading_en_mxh', 'shading_en_string_option', 'shading_en_timestep', 'shading_string_option'])

In [68]:
generation["Shading"]["shading_en_azal"]

0.0

In [12]:
sum(generation["poa"])

1729310.0062196068

In [10]:
generation["solrad_annual"]

4.731790645496261

In [60]:
sum(generation["poa_monthly"])

1729.3100062196077

In [58]:
sum(generation["solrad_monthly"])/12

4.731790645496262

In [102]:
sum(generation["dc_monthly"])

39607.09450683009

In [87]:
days = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]
from itertools import accumulate
cumulative_sum = list(accumulate(days))
cumulative_sum

[0, 31, 59, 90, 120, 151, 181, 212, 243, 273, 304, 334, 365]

In [99]:
DCOut = np.array(generation["dc"]).reshape(365, 24)
averages = np.zeros((12,24))
for month in range(12):
    for hour in range(24):
        averages[month,hour] = np.average(DCOut[cumulative_sum[month]:cumulative_sum[month+1],hour])
averages = averages.reshape(12*24)

array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 9.78394081e+01,
       1.93906492e+03, 4.82156395e+03, 9.34980015e+03, 1.06828922e+04,
       1.05325578e+04, 9.40722935e+03, 6.50660864e+03, 2.51250333e+03,
       3.59878346e+02, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.69040672e+03,
       5.18679054e+03, 8.22156695e+03, 1.29343213e+04, 1.40401744e+04,
       1.32676365e+04, 1.13635526e+04, 8.08107566e+03, 4.44322937e+03,
       1.09905491e+03, 8.04986908e+01, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 4.19173430e+02, 3.14859153e+03,
      

# As a function

In [12]:
import json
import pandas as pd
import numpy as np
import PySAM.Pvwattsv8 as PVWatts
import PySAM.Grid as Grid
import csv
import os
import geopandas as gpd
import math

In [13]:
def loadModules(file_names):
    pv = PVWatts.new()
    grid = Grid.from_existing(pv)

    with open(file_names[0], 'r') as file:
        data = json.load(file)
        for k, v in data.items():
                pv.value(k, v)

    with open(file_names[1], 'r') as file:
        data = json.load(file)
        for k, v in data.items():
                grid.value(k, v)

    return pv, grid

def get_matrix(tilts):
    tilts = tilts.round()
    angles = np.unique(tilts)
    azimuths = np.arange(0, 360, 1)

    matrix = []
    singleRow = []
    singleRow.append(0)
    for angle in azimuths:
        singleRow.append(angle)

    matrix.append(singleRow)
    for i, angle in enumerate(angles):
        singleRow = []
        singleRow.append(angle)
        for j, tilt in enumerate(tilts):
            if angle <= tilt:
                singleRow.append(100)
            else:
                singleRow.append(0)
        matrix.append(singleRow)
    return matrix


def runPySAMSimulation(file_names, tilts, plane, tmyfile, exportPath):
    pv, grid = loadModules(file_names)
    shadingMatrix = get_matrix(tilts)

    area, tilt, azimuth = getInfoRoof(plane)
    
    ratio=0.400/(1.879*1.045)

    modifiedParams = {"shading_azal": shadingMatrix,
        "system_capacity": area*ratio, #*self.pv.value("gcr"), #We don't need the area by the ground coverage ratio
        "tilt": tilt,
        "azimuth": azimuth,
        "solar_resource_file": tmyfile}

    for i in range(len(modifiedParams)): 
        pv.value(list(modifiedParams.keys())[i], list(modifiedParams.values())[i])

    modules = [pv, grid]
    
    for m in modules:
        m.execute()

    generation = pv.export()["Outputs"]["ac"]
    generation = np.array(generation).reshape(365, 24)
    generation_df = pd.DataFrame(generation)
    
    # generation_df.to_csv((exportPath + '.csv'), index=False)
    return generation_df/area



In [14]:
file_names = ["/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/Scripts/sunEstimation/pysam_template_pvwattsv8.json",
    "/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/Scripts/sunEstimation/pysam_template_grid.json"]

tmyfile = "/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/RAW_Data/TMY/NREL/419806_41.41_2.22_tmy-2022.csv"

basePath = "/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3/"
neighborhood = "Test_70_el Besòs i el Maresme"
parcelsFolder = basePath + "/Results/" + neighborhood + "/Parcels/"

annual_ac = 0

for parcel in os.listdir(parcelsFolder):
    parcelSubfolder = parcelsFolder + parcel + "/"
    for construction in [x for x in os.listdir(parcelSubfolder) if os.path.isdir(parcelSubfolder + x)]:
        constructionFolder = parcelSubfolder + construction + "/"
        planesGDF = gpd.read_file(constructionFolder + "Plane Identification/" + construction + ".gpkg")
        for cluster in planesGDF.cluster.values:
            shadingFile = constructionFolder + "/Shading/" + str(cluster) + ".csv"
            plane = planesGDF[planesGDF.cluster == cluster]
            if os.path.isfile(shadingFile):
                if(os.stat(shadingFile).st_size > 0):
                    shadingProfilesDF = pd.read_csv(shadingFile, header=None)
                    tilts = shadingProfilesDF.iloc[0][3:363]

                    generation_df = runPySAMSimulation(file_names, tilts, plane, tmyfile, "exportPath")
                    annual_ac = generation_df.sum().sum()        
print(annual_ac)

272938.68342764745


In [36]:
shadingFile

'/home/jaumeasensio/Documents/Projectes/BEEGroup/solar_potencial_estimation_v3//Results/Test_70_el Besòs i el Maresme/Parcels/4151302DF3845A/139//Shading/12.csv'